In [ ]:
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np

In [ ]:
%%capture
!apt-get install libav-tools -y

In [ ]:
from fastai.vision import *
import librosa
import librosa.display
import pylab
import matplotlib
import gc

In [ ]:
!mkdir /kaggle/working/train
!mkdir /kaggle/working/test

In [ ]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/kaggle/working/train/' + name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [ ]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/kaggle/working/test/' + name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [ ]:
Data_dir=np.array(glob("../input/train/Train/*"))

In [ ]:
%%memit 
i=0
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

In [ ]:
gc.collect()

In [ ]:
%%memit 
i=1500
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

In [ ]:
gc.collect()

In [ ]:
%%memit 
i=3000
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

In [ ]:
gc.collect()

In [ ]:
%%memit 
i=4500
for file in Data_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

In [ ]:
gc.collect()

In [ ]:
path = Path('/kaggle/working/')
np.random.seed(42)
data = ImageDataBunch.from_csv(path,csv_labels='../input/train.csv', folder="train", valid_pct=0.2, suffix='.jpg',
        ds_tfms=get_transforms(), size=224, num_workers=0).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-3))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

In [ ]:
learn.save('stage-3')

In [ ]:
learn.export()